# A multiperiod production model

In [1]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.


Another common way in which models are expanded is by
replicating them over time.  To illustrate, we
consider how the model of Figure @Tut1@-@steel.mod@
might be used to plan production for
the next $T$ weeks, rather than for a single week.

We begin by adding another index set to most of the quantities of
interest.  The added set represents weeks numbered 1 through
$T$, as shown in Figure @Tut4@-@steelT0.mod@.


steelT0.mod
Figure @Tut4@-@steelT0.mod@
Production model replicated over periods (\f(CWsteelT0.mod\fP).
file~[steelT0.mod]

In [2]:
%%writefile steelT0.mod

set PROD;     # products
param T > 0;  # number of weeks

param rate {PROD} > 0;         # tons per hour produced
param avail {1..T} >= 0;       # hours available in week
param profit {PROD,1..T};      # profit per ton
param market {PROD,1..T} >= 0; # limit on tons sold in week

# tons produced
var Make {p in PROD, t in 1..T} >= 0, <= market[p,t];
                   
# total profits from all products in all weeks
maximize Total_Profit:
   sum {p in PROD, t in 1..T} profit[p,t] * Make[p,t];
    
# total of hours used by all products
# may not exceed hours available, in each week
subject to Time {t in 1..T}:
   sum {p in PROD} (1/rate[p]) * Make[p,t] <= avail[t];

Writing steelT0.mod


The expression
`1..T`
is `AMPL`'s shorthand for the set of integers from 1
through
`T` .
We have replicated all the parameters and variables over
this set, except for
`rate` ,
which is regarded as fixed over time.  As a
result there is a constraint for each week, and the
`profit`
terms are
summed over weeks as well as products.

So far this is merely a separate LP for each week,
unless something is added to tie the weeks together.  Just as we were able
to find constraints that involved all the products, we could look for
constraints that involve production in all of the weeks.  Most multiperiod
models take a different approach, however, in which constraints relate
each week's production to that of the following week only.

Suppose that we allow some
of a week's production to be placed in inventory, for sale in any later week.
We thus add new decision variables to represent the amounts inventoried
and sold in each week.  The variables
`Make[j,t]`
are retained, but they
represent only the amounts produced, which are now not necessarily the
same as the amounts sold.  Our new variable declarations look like this:

```
var Make {PROD,1..T} >= 0;
var Inv {PROD,0..T} >= 0;

var Sell {p in PROD, t in 1..T} >= 0, <= market[p,t];
```

The bounds
`market[p,t]` ,
which represent the maximum amounts that can
be sold in a week, are naturally transferred to
`Sell[p,t]` .

The variable
`Inv[p,t]`
will represent the inventory of product
`p`
at the
end of period
`t` .
Thus the quantities
`Inv[p,0]`
will be the inventories at the
end of week zero, or equivalently at the beginning of the first week 
&ndash; 
in
other words, now.  Our model assumes that these initial inventories are
provided as part of the data:

```
param inv0 {PROD} >= 0;
```

A simple constraint guarantees that the variables
`Inv[p,0]`
take these
values:

```
subject to Init_Inv {p in PROD}:  Inv[p,0] = inv0[p];
```

It may seem "inefficient" to devote a constraint like this to saying that a
variable equals a constant, but when it comes time to send the linear
program to a solver, `AMPL` will
automatically substitute the value of
`inv0[p]`
for any occurrence of
`Inv[p,0]` .
In most cases, we can concentrate on writing the model in the clearest or
easiest way, and leave matters of efficiency to the computer.

Now that we are distinguishing sales, production, and inventory, we
can explicitly model the contribution of each to the profit, by defining three
parameters:

```
param revenue {PROD,1..T} >= 0;
param prodcost {PROD} >= 0;
param invcost {PROD} >= 0;
```

These are incorporated into the objective as follows:

```
maximize Total_Profit:
   sum {p in PROD, t in 1..T} (revenue[p,t]*Sell[p,t] -
      prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
```

As you can see,
`revenue[p,t]`
is the amount received per ton of product
`p`
sold in week
`t` ;
`prodcost[p]`
and
`invcost[p]`
are the production and inventory
carrying cost per ton of product
`p`
in any week.

Finally, with the sales and inventories fully incorporated into our
model, we can add the key constraints that tie the weeks together:
the amount of a product made available in a week,
through production or from inventory, must equal the amount disposed of
in that week, through sale or to inventory:

```
subject to Balance {p in PROD, t in 1..T}:
   Make[p,t] + Inv[p,t-1] = Sell[p,t] + Inv[p,t];
```

Because the index
`t`
is from a set of numbers, the period
previous to
`t`
can be written as
`t-1` .
In fact,
`t`
can be used in any arithmetic
expression;
conversely, an `AMPL` expression such as
`t-1`
may be used in
any context where it makes sense.  Notice also that for a first-period
constraint
(`t`
equal to 1), the inventory term on the left is
`Inv[p,0]` ,
the
initial
inventory.

We now have a complete model, as shown in Figure @Tut4@-@steelT.mod@.
To illustrate a solution, we use the small sample data file
shown in Figure @Tut4@-@steelT.dat@; it
represents a four-week expansion of the data from Figure @Tut1@-@steel.dat@.

 .1i
.get steelT.mod

.C "Figure @Tut4@-@steelT.mod@" "Multiperiod production model (\f(CWsteelT.mod\fP).
.SP

.get steelT.dat

.C "Figure @Tut4@-@steelT.dat@" "Data for multiperiod production model (\f(CWsteelT.dat\fP).
.ix file~[steelT.mod],~[steelT.dat]


In [3]:
%%writefile steelT.mod

set PROD;     # products
param T > 0;  # number of weeks

param rate {PROD} > 0;          # tons per hour produced
param inv0 {PROD} >= 0;         # initial inventory
param avail {1..T} >= 0;        # hours available in week
param market {PROD,1..T} >= 0;  # limit on tons sold in week

param prodcost {PROD} >= 0;     # cost per ton produced
param invcost {PROD} >= 0;      # carrying cost/ton of inventory
param revenue {PROD,1..T} >= 0; # revenue per ton sold

var Make {PROD,1..T} >= 0;      # tons produced
var Inv {PROD,0..T} >= 0;       # tons inventoried
var Sell {p in PROD, t in 1..T} >= 0, <= market[p,t]; # tons sold

# Total revenue less costs in all weeks
maximize Total_Profit:
   sum {p in PROD, t in 1..T} (revenue[p,t]*Sell[p,t] -
      prodcost[p]*Make[p,t] - invcost[p]*Inv[p,t]);
           
# Total of hours used by all products
# may not exceed hours available, in each week
subject to Time {t in 1..T}:
   sum {p in PROD} (1/rate[p]) * Make[p,t] <= avail[t];

# Initial inventory must equal given value
subject to Init_Inv {p in PROD}:  Inv[p,0] = inv0[p];

# Tons produced and taken from inventory
# must equal tons sold and put into inventory
subject to Balance {p in PROD, t in 1..T}:
   Make[p,t] + Inv[p,t-1] = Sell[p,t] + Inv[p,t];

Overwriting steelT.mod


In [4]:
T = 4

avail =  {1: 40,  2: 40,  3: 32,  4: 40}

df_prod = pd.DataFrame(
    [
        ['bands', 200, 10, 10, 2.5],
        ['coils', 140, 0, 11, 3]
    ],
    columns = ['PROD', 'rate', 'inv0', 'prodcost', 'invcost']
).set_index('PROD')

df_info = pd.DataFrame(
    [
        ['bands', 1, 25, 6000],
        ['bands', 2, 26, 6000],
        ['bands', 3, 27, 4000],
        ['bands', 4, 27, 6500],
        ['coils', 1, 30, 4000],
        ['coils', 2, 35, 2500],
        ['coils', 3, 37, 3500],
        ['coils', 4, 39, 4200]
    ],
    columns = ['PROD', 'T', 'revenue', 'market']
).set_index(['PROD', 'T'])

display(df_prod)
display(df_info)

,rate,inv0,prodcost,invcost
PROD,,,,
bands,200,10,10,2.5
coils,140,0,11,3.0


revenue  market
PROD  T                 
bands 1       25    6000
      2       26    6000
      3       27    4000
      4       27    6500
coils 1       30    4000
      2       35    2500
      3       37    3500
      4       39    4200

If we put the model and data into files
`steelT.mod
and
`steelT.dat ,
then `AMPL` can be invoked to find a solution:

In [15]:
ampl = AMPL()
ampl.read('steelT.mod')

ampl.param['T'] = T
ampl.param['avail'] = avail

ampl.set_data(df_prod, 'PROD')
ampl.set_data(df_info)

ampl.option['solver'] = 'highs'
ampl.solve()

#ampl.option['display_1col'] = 0
ampl.display('Make')

make = ampl.var['Make'].to_pandas()
make.columns = ["Make"]
make.index.names = ['PROD', 'T']
display(make)

make = make.unstack()
display(make)
display(make.T)

HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 515033
16 simplex iterations
0 barrier iterations
 
Make :=
bands 1   5990
bands 2   6000
bands 3   1400
bands 4   2000
coils 1   1407
coils 2   1400
coils 3   3500
coils 4   4200
;



Make
PROD  T      
bands 1  5990
      2  6000
      3  1400
      4  2000
coils 1  1407
      2  1400
      3  3500
      4  4200

Make                  
T         1     2     3     4
PROD                         
bands  5990  6000  1400  2000
coils  1407  1400  3500  4200

PROD    bands  coils
     T              
Make 1   5990   1407
     2   6000   1400
     3   1400   3500
     4   2000   4200

Production of coils in the first week is held over to be sold at a
higher price in the second week.  In the second through fourth weeks,
coils are more
profitable than bands, and so coils are sold up to the limit, with
bands filling out the capacity.
(Setting option
`display_1col`
to zero permits this output to appear in a nicer format, as explained in
Section @Display@.@disp-format@.)